# Question Answering

In this notebook we are training two RNNs: one to process a story and another one to process a question. The hidden states of both networks are then merged and used to classify a single word answer. The [bAbI](https://research.facebook.com/researchers/1543934539189348) dataset is going to be used for this setup.

This model is implemented as a Keras [baseline QA model](https://github.com/fchollet/keras/blob/master/examples/babi_rnn.py) and based on the original paper "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks" (Weston et al. 2015).

In [ ]:
from __future__ import print_function
from functools import reduce
import re
import tarfile

import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge, Dropout, RepeatVector
from keras.layers import recurrent
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

First build auxiliar functions to tokenize and parse the stories from bAbI dataset

In [ ]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [ ]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [ ]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories, and then convert the sentences into a single story.
    If max_length is supplied, any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

Now make vectorization function, as in previous notebooks, to map words to indeces and add necessary padding.

In [ ]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        y = np.zeros(len(word_idx) + 1)  # let's not forget that index 0 is reserved
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return pad_sequences(X, maxlen=story_maxlen), pad_sequences(Xq, maxlen=query_maxlen), np.array(Y)

Define some network parameters and get the actual data from S3, and then vecotrize

In [ ]:
RNN = recurrent.GRU
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN, EMBED_HIDDEN_SIZE, SENT_HIDDEN_SIZE, QUERY_HIDDEN_SIZE))

try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise

In [ ]:
tar = tarfile.open(path)
# Default QA1 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
# challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

vocab = sorted(reduce(lambda x, y: x | y, (set(story + q + [answer]) for story, q, answer in train + test)))
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

X, Xq, Y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tX, tXq, tY = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)

print('vocab = {}'.format(vocab))
print('X.shape = {}'.format(X.shape))
print('Xq.shape = {}'.format(Xq.shape))
print('Y.shape = {}'.format(Y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

**Exercise:** Build the Question sequence encoder. Remember to place an Embedding layer of output `EMBED_HIDDEN_SIZE` with its input to `query_maxlen`. Also, the hidden state $h_t$ has to be replicated to match the length of story_maxlen.

In [ ]:
# TODO: Build the Question RNN encoder. 
qrnn = Sequential()
qrnn.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE,
                   input_length=query_maxlen))
qrnn.add(Dropout(0.3))
qrnn.add(RNN(EMBED_HIDDEN_SIZE, return_sequences=False))
qrnn.add(RepeatVector(story_maxlen))

**Exercise:** Build the sentence encoder, merge it with the previously built question encoder, and stack the classification output on top of it (just one word as output). Remember to place an Embedding layer of output `EMBED_HIDDEN_SIZE` with sequence length `story_maxlen` for the story RNN. 

In [ ]:
# TODO: Build the Story Embedding input
sentrnn = Sequential()
# ...

# TODO: Merge the story Embeddings to the question encoder RNN, and then place softmax classification
model = Sequential()
# ...

# TODO: compile the model with classification objective, requesting accuracy
# ...


In [ ]:
# check model construction
model.summary()

In [ ]:
print('Training..')
his = model.fit([X, Xq], Y, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, validation_split=0.05, verbose=2)

In [ ]:
loss, acc = model.evaluate([tX, tXq], tY, batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

In [ ]:
# Define function to perform question and answer [story + query]
def ask_qa(story, query):
    print('-' * 15) 
    t_story = pad_sequences([[word_idx[i] for i in tokenize(story)]], maxlen=story_maxlen)
    t_query = pad_sequences([[word_idx[i] for i in tokenize(query)]], maxlen=query_maxlen)

    # fill the rest of batch with paddings to fit into the same model defined to train
    story_batch = t_story
    query_batch = t_query

    preds = model.predict_on_batch([story_batch, query_batch])
    #print('preds shape: ', preds.shape)
    probs = preds[0]
    #print('probs: ', probs)
    response = np.argmax(probs)
    print('S:{}\nQ:{}\nA:{}'.format(story, query, idx_word[response]))

story1 = 'John went to the bathroom.'
query1 = 'Where is John?'
ask_qa(story1, query1)

story2 = 'Mary went to the garden.'
query2 = 'Where is Mary?'
ask_qa(story2, query2)

story3 = 'John moved to the kitchen.'
query3 = 'Where is John?'
ask_qa(story3, query3)

story4 = 'John went to the kitchen. Daniel moved to the garden.'
query4 = 'Where is Daniel?'
ask_qa(story4, query4)